In [1]:
import numpy as np
import matplotlib.pyplot as plt
from qutip import  *

In [31]:
n = 2
vac = basis(n, 0) # vacuum state in Fock space for a two-leve spin case (1/2)
a = destroy(n)
ad = create(n)
print(vac)
print(a)

ad*vac

Quantum object: dims=[[2], [1]], shape=(2, 1), type='ket', dtype=Dense
Qobj data =
[[1.]
 [0.]]
Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dia, isherm=False
Qobj data =
[[0. 1.]
 [0. 0.]]


Quantum object: dims=[[2], [1]], shape=(2, 1), type='ket', dtype=Dense
Qobj data =
[[0.]
 [1.]]

In [32]:
10 * sigmaz()

Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=CSR, isherm=True
Qobj data =
[[ 10.   0.]
 [  0. -10.]]